In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Załóżmy, że mamy dane treningowe: X_train - cechy, y_train - etykiety (zakłócone), noise_rates - znane poziomy zakłóceń

# Tworzymy tensor z danymi treningowymi
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)

# Tworzymy klasę naszej sieci neuronowej
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

# Inicjalizujemy naszą sieć
input_dim = X_train.shape[1]
hidden_dim = 100
output_dim = len(np.unique(y_train))
net = NeuralNet()

# Definiujemy funkcję straty (estymator bezstronny)
def unbiased_loss(logits, targets, noise_rates):
    loss = nn.CrossEntropyLoss(reduction='none')
    losses = loss(logits, targets)
    unbiased_losses = losses * (1 - noise_rates[targets])
    return torch.mean(unbiased_losses)

# Definiujemy optymalizator i parametry szkolenia
optimizer = optim.Adam(net.parameters(), lr=0.001)
num_epochs = 100
batch_size = 32

# Szkolenie modelu
for epoch in range(num_epochs):
    # Dzielimy dane na mini-batche
    permutation = torch.randperm(X_train_tensor.size()[0])
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

        # Wyzerowanie gradientów
        optimizer.zero_grad()

        # Przekazanie danych przez sieć
        logits = net(batch_x)

        # Obliczenie straty i propagacja wsteczna
        loss = unbiased_loss(logits, batch_y, noise_rates)
        loss.backward()
        optimizer.step()

    # Obliczenie i wydruk straty w każdej epoce
    with torch.no_grad():
        logits = net(X_train_tensor)
        loss = unbiased_loss(logits, y_train_tensor, noise_rates)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testowanie modelu na danych testowych
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.int64)
    logits = net(X_test_tensor)
    predicted_labels = torch.argmax(logits, dim=1)
    accuracy = torch.sum(predicted_labels == y_test_tensor).item() / len(y_test)
    print(f'Accuracy on test set: {accuracy:.4f}')
